<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="Logo HEIG-VD" style="width: 80px;" align="right"/>

# Cours APN - Labo 3 : Groupement hiérarchique de documents

## Résumé
Le but de ce laboratoire est d'appliquer un algorithme de groupement hiérarchique à des résumés de films.  Vous afficherez les résultats et observerez quels paramètres permettent d'obtenir un groupement convenable, car il n'y a pas de vérité-terrain dans ces données.

Les étapes proposées sont les suivantes :
  1. Préparation des documents : pré-traitement des textes des résumés
  2. Création des représentations vectorielles de chaque résumé
  3. Groupement hiérarchique, affichage du dendrogramme, et discussion

## 1. Préparation des documents

Dans ce labo, vous utiliserez les titres et résumés des 100 films classiques (*référence anonymisée*) qui vous sont fournis sur Cyberlearn dans le fichier `movie_data.csv` sur deux colonnes : titres et résumés.

Dans cette première partie, vous allez transformer les textes des résumés pour réduire la diversité du vocabulaire.  Un ensemble de fonctions de préparation vous est fourni dans la librairie `utils.py` (également sur Cyberlearn) et il est conseillé de les lire et de les utiliser.

In [36]:
import pandas as pd
import utils # librairie avec des fonctions de pré-traitement des textes
# elle fait appel à la librairie NLTK (ce qui affiche plusieurs messages)

movie_data = pd.read_csv("movie_data.csv")

a. Veuillez charger les données des films depuis le fichier `movie_data.csv` dans une _DataFrame_ intitulée `movie_data`.  Combien de lignes et colonnes a-t-elle et quels sont les intitulés des colonnes ?

In [9]:
movie_data.shape
movie_data.columns

Index(['Title', 'Synopsis'], dtype='object')

b. Veuillez normaliser le texte de chaque résumé en appliquant plusieurs des fonctions fournies dans `utils.py` (pour cela, vous devez examiner ce fichier).  Veuillez stocker les textes normalisés dans une nouvelle colonne de la _DataFrame_.   Il n'est pas nécessaire de tokeniser les textes normalisés, car le _vectorizer_ de sklearn le fera plus loin. 

In [37]:
movie_data['normalized_text'] = movie_data['Synopsis'].apply(utils.lemmatize_text)
movie_data['normalized_text'] = movie_data['Synopsis'].apply(utils.remove_punctuation)
movie_data['normalized_text'] = movie_data['Synopsis'].apply(utils.remove_stopwords)
movie_data['normalized_text'] = movie_data['Synopsis'].apply(utils.remove_non_alphabetic)


In [23]:
movie_data['normalized_text']

0     In late summer guests are gathered for the wed...
1     In Andy Dufresne Tim Robbins a banker in Maine...
2     The relocation of Polish Jews from surrounding...
3     The film opens in where an older and fatter Ja...
4     In the early years of World War II December th...
                            ...                        
95    Shortly after moving to Los Angeles with his p...
96    L.B Jeff Jeffries James Stewart recuperates fr...
97    Sights of Vienna Austria flash across the scre...
98    At the end of an ordinary work day advertising...
99    A film of the life of the renowned musical com...
Name: normalized_text, Length: 100, dtype: object

c. Veuillez analyser la longueur des résumés, avant et après le pré-traitement, grâce au _5-number summary_ de chaque distribution.  Veuillez commenter brièvement l'évolution de la distribution suite au pré-traitement.

In [25]:
movie_data['length_before'] = movie_data['Synopsis'].apply(len)
movie_data['length_after'] = movie_data['normalized_text'].apply(len)

In [32]:
movie_data['length_before'].describe()


count      100.000000
mean      8779.840000
std       6603.462185
min       1071.000000
25%       4378.750000
50%       6528.500000
75%      11417.500000
max      34021.000000
Name: length_before, dtype: float64

In [33]:
movie_data['length_after'].describe()

count      100.000000
mean      8586.100000
std       6465.866851
min       1040.000000
25%       4284.250000
50%       6406.000000
75%      11204.250000
max      33305.000000
Name: length_after, dtype: float64

d. Veuillez afficher un extrait du résumé d'un film *avant* et *après* le pré-traitement et commenter brièvement le résultat.

In [38]:
movie_data['normalized_text'][0]

"In late summer guests are gathered for the wedding reception of Don Vito Corleone 's daughter Connie Talia Shire and Carlo Rizzi Gianni Russo Vito Marlon Brando the head of the Corleone Mafia family is known to friends and associates as Godfather He and Tom Hagen Robert Duvall the Corleone family lawyer are hearing requests for favors because according to Italian tradition no Sicilian can refuse a request on his daughter 's wedding day One of the men who asks the Don for a favor is Amerigo Bonasera a successful mortician and acquaintance of the Don whose daughter was brutally beaten by two young men because she refused their advances the men received minimal punishment The Don is disappointed in Bonasera who 'd avoided most contact with the Don due to Corleone 's nefarious business dealings The Don 's wife is godmother to Bonasera 's shamed daughter a relationship the Don uses to extract new loyalty from the undertaker The Don agrees to have his men punish the young men responsible.Me

## 2. Création des représentations vectorielles

Dans cette partie, vous allez créer pour chaque film une représentation vectorielle avec les coefficients TF-IDF, comme vous l'avez appris en PCD, en utilisant `TfidfVectorizer`.  Veuillez justifier brièvement votre choix de paramètres.

a. Veuillez créer les vecteurs dans un tableau appelé `feature_matrix`, avec autant de lignes que de films.

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [62]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 

feature_matrix = tfidf_vectorizer.fit_transform(movie_data['normalized_text'])



b. Quel est le nombre de colonnes du tableau `feature_matrix`. Pourquoi ?

In [63]:
feature_matrix.shape

(100, 13789)

c. Veuillez afficher les mots correspondant aux 20 premières dimensions de `feature_matrix`.  (Note : les dimensions sont classées par ordre alphabétique des mots.)

In [72]:
feature_names = tfidf_vectorizer.get_feature_names_out()

feature_names[:20]

array(['000', '00am', '101st', '11th', '12', '120', '1300', '13th', '14',
       '15', '15th', '16', '16th', '17', '18', '1800s', '1880', '18th',
       '1900s', '1920s'], dtype=object)

## 3. Groupement hiérarchique
Dans cette section, vous utiliserez un algorithme de groupement hiérarchique et vous afficherez les résultats sous forme de dendrogramme.  Les fonctions utilisées proviendront de `scipy`, comme recommandé par sklearn sous [Plot Hierarchical Clustering Dendrogram](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html).  Merci de suivre les indications et répondre aux questions suivantes.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import single, complete, average, ward
from scipy.cluster.hierarchy import dendrogram, cut_tree
import matplotlib.pyplot as plt
%matplotlib inline

a. Générer une matrice des distances entre paires de résumés, en utilisant les représentations vectorielles obtenues ci-dessus et la métrique [sklearn.metrics.pairwise.cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html).  Attention, le cosinus étant une similarité, prendre _1-cosinus_ pour obtenir des distances. 

b. Veuillez utiliser la méthode **ward** de [scipy.cluster.hierarchy](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html) qui effectue le groupement hiérarchique à partir d'une matrice des distances.  Quel est le résultat retourné par la méthode ?

c. Afficher le dendrogramme avec la fonction [scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) qui utilise la matrice de liens.  Utiliser l'exemple donné en bas de la page [dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html).  Veuillez afficher un dendrogramme aussi lisible que possible, et en particulier veuillez étiqueter chaque feuille avec le nom du film correspondant.

d. Veuillez essayer une deuxième méthode de groupement hiérarchique et afficher son dendrogramme.  Comment se compare-t-il au précédent ?  Le trouvez-vous meilleur, moins bon, ou similaire ? 

e. Quel est le nombre de groupes qui vous semble optimal, pour la méthode de votre choix ?  Pourquoi ?

## 4. Affichage des groupes et de leurs mots-clés

Dans cette section, vous utiliserez la méthode de groupement et le nombre de groupes choisis dans la section précédente.  Pour obtenir un nombre de groupe fixé à l'avance à partir de la matrice *linkage* obtenue au 3b ou 3d, vous pouvez utiliser la fonction `cut_tree`de Scipy. 

a. Pour chaque groupe, veuillez afficher :
* le numéro du groupe (commençant par 1)
* les titres des films du groupe
* le nombre de films dans le groupe

b. Pour chacun des groupes précédents, veuillez afficher les six meilleurs mots-clés qui le caractérisent.  On définit les mots-clés comme les mots (donc les dimensions résultant du `TfidfVectorizer`) qui ont les plus grands coefficients dans le centre de gravité du groupe.

c. D'après les titres des films (et vos connaissances), les mots-clés obtenus sont-ils représentatifs des groupes ?

**Fin du Labo 3.**  Veuillez nettoyer ce notebook, afficher les résultats et les commentaires demandés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.